In [ ]:
import numpy as np
import pandas as pd

# 1. System Parameters (Linear Dynamics)
# s_{t+1} = A*s_t + B*a_t
A = np.array([[1.1, 0], [0, 1.0]])
B = np.array([[0.5, 0], [0, 1.0]])
U = np.eye(2)    # State cost
V = 0.1 * np.eye(2) # Action cost
T = 10           # Time horizon

# 2. LQR Solver via Riccati Recursion
def solve_lqr(A, B, U, V, T):
    # Initialize Phi at terminal time T [cite: 4098]
    phi = -U
    phis = [phi]
    Ls = []
    
    # Backward recursion for t = T-1 down to 0 [cite: 4154]
    for t in range(T-1, -1, -1):
        # Calculate optimal gain matrix L_t [cite: 4120-4123]
        # L = -(B^T * Phi * B + V)^-1 * B^T * Phi * A
        term_inv = np.linalg.inv(B.T.dot(phi).dot(B) - V)
        L = term_inv.dot(B.T).dot(phi).dot(A)
        
        # Update Phi for time t using Discrete-Time Riccati Equation
        # Phi_t = A^T * Phi_{t+1} * (A + B*L) - U
        phi = A.T.dot(phi).dot(A + B.dot(L)) - U
        
        phis.append(phi)
        Ls.append(L)
        
    return Ls[::-1], phis[::-1]

# 3. Execution
gains, value_matrices = solve_lqr(A, B, U, V, T)

# 4. Controller Inference
current_state = np.array([1.0, 0.5]) # Initial deviation
optimal_action = gains[0].dot(current_state)

print(f"Optimal Gain Matrix (L_0):\n{gains[0]}")
print(f"Optimal Action for current state: {optimal_action}")